In [204]:

import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.init as init
import torch.optim as optim
import random

In [205]:
width,height=4,4

In [206]:
import os,sys,inspect
sys.path.insert(1, os.path.join(sys.path[0], '..'))

In [207]:
%load_ext autoreload
%autoreload 2
import env.env as env
import numpy as np

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [208]:
value_approximator=nn.Sequential(nn.Linear(16,4),nn.Linear(4,1))
states = [[i, j] for i in range(4) for j in range(4)]

In [209]:
state_tensor=torch.tensor(grid.current_state.flatten()[:]).float()

In [210]:
loss=nn.MSELoss()

In [211]:
optimizer = optim.SGD(value_approximator.parameters(), lr=0.001)

In [212]:
#One Episodestartx=random.randint(0,3)
grid=env.Grid_game(height,width)
Counter=np.zeros((width,height))
#starty=0
#startx=3
history=[]
Returns=np.zeros((height,width))
rewards=0
k=0.7

y,x=grid.player.curpos

num_episodes=100


for i in range(num_episodes):
    state_list=[]
    starty,startx = random.choice(states[1:-1])
    grid.set_pos(starty,startx)
    iteration=0
    rewards=0
 #   if num_episodes % 1000:
      #  print(mse_loss)
    while(not grid.state):
        value_approximator.zero_grad()
        
        oldpos=grid.player.curpos[:]
      #  oldstate=torch.tensor(oldpos).float()
        oldstate=torch.tensor(grid.current_state.flatten()).float()
        iteration+=1
        state,cur_state,reward,_=grid.transition()
        curpos=grid.player.curpos[:]
        newstate=torch.tensor(grid.current_state.flatten()).float()
        Counter[curpos[0]][curpos[1]]+=1
        
        val_newstate=-1+k*value_approximator(newstate)
        val_oldstate=value_approximator(oldstate)
        
        mse_loss=loss(val_oldstate,val_newstate)
        mse_loss.backward()
        optimizer.step()
        
    history.append(grid.vst[:].copy())
    
    

In [214]:
for i in range(4):
    for j in range(4):
        if i==0 and j==0:
            continue
        if i==3 and j==3:
            continue
        grid.set_pos(i,j)
        print(grid.current_state)
        grid.vst[i][j]=value_approximator(torch.tensor(grid.current_state.flatten()).float())

[[-1.  1.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0. -1.]]
[[-1.  0.  1.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0. -1.]]
[[-1.  0.  0.  1.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0. -1.]]
[[-1.  0.  0.  0.]
 [ 1.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0. -1.]]
[[-1.  0.  0.  0.]
 [ 0.  1.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0. -1.]]
[[-1.  0.  0.  0.]
 [ 0.  0.  1.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0. -1.]]
[[-1.  0.  0.  0.]
 [ 0.  0.  0.  1.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0. -1.]]
[[-1.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 1.  0.  0.  0.]
 [ 0.  0.  0. -1.]]
[[-1.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  1.  0.  0.]
 [ 0.  0.  0. -1.]]
[[-1.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  1.  0.]
 [ 0.  0.  0. -1.]]
[[-1.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  1.]
 [ 0.  0.  0. -1.]]
[[-1.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 1.  0.  0. -1.]]
[[-1.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  1.  0. -1.]]

In [215]:
grid.vst

array([[ 0.        , -1.31682181, -1.31258655, -1.18911457],
       [-1.18050098, -1.25915074, -1.24770808, -1.20301569],
       [-1.14106941, -1.2423799 , -1.31728566, -1.09882414],
       [-1.00070858, -1.15924954, -1.13227201,  0.        ]])

In [151]:
torch.cat([torch.tensor(grid.initial_board.flatten()).float(),torch.tensor([3,4]).float()])

tensor([-1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0., -1.,  3.,  4.])

In [197]:
grid.current_state

array([[0., 0., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])